In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
from itertools import combinations
import gensim
from gensim.models import KeyedVectors

# Parameters Selection for NMF

We present the more advanced task of parameters selection for our NMF topic modelling - namely, selecting the optimal combination (*K*,$\alpha$,*$l_1$*).

\begin{align}
\label{eq:1}
\min_{W,H \ge 0} \frac{1}{2} ||X-WH||_{Fro}^2 + \alpha \left[ l_1 \left( ||W||_1 + ||H||_1 \right) + \left( 1-l_1 \right) \left(||W||_{Fro}^2 + ||H||_{Fro}^2  \right) \right]
\end{align}


### Download data and models

First, let's load the TF-IDF normalised document-term matrix and list of terms that we stored earlier using *Joblib*:

In [3]:
(tfidf, tfidf_feature_names) = joblib.load("./interm/tfidf_matrix-features_names.pkl" )
print( "Loaded %d X %d document-term matrix" % (tfidf.shape[0], tfidf.shape[1]) )

Loaded 4691 X 1300 document-term matrix


Then load the topic models for different triplets  (k, $\alpha$, $l_1$)

In [4]:
topic_models = joblib.load("./interm/topic_models.pkl") 
print( "Loaded %d topic models" % (len(topic_models)) )

Loaded 16731 topic models


Finally load Word2vec model

In [5]:
w2v_model = KeyedVectors.load("./interm/word2vec.wordvectors", mmap='r')

### Select the triplet (K, $\alpha$, $l_1$)

We use the Word2vec model as part of our *topic coherence* approach to evaluate the different NMF topic models that we created previously. To do this, we implement a simple version of the TC-W2V coherence measure.

We define the coherence score of each topic $k$, $TCW2V_k$ as follows:

\begin{align}
TCW2V_k = \frac{1}{\dbinom{N}{2}} \sum_{j=2}^N \sum_{i=1}^{j-1} similarity\left(wv_{tj}, wv_{ti}\right)
\end{align}

Where $N=5$ is the number of terms charcterizing each topic, $wv_{kj}$ is the vector associated to term $j$ characterizing topic $k$, $wv_{ki}$ is the vector associated to term $i$ characterizing topic $k$ and $similarity(A,B)$ is the cosine similarity of vectors $A$ and $B$. 

The **total coherence score of a NMF model** is the **average coherence score of all topics**.

In [6]:
def calculate_coherence(w2v_model, term_rankings):
    overall_coherence = 0.0
    for topic_index in range(len(term_rankings)):
        # check each pair of terms
        pair_scores = []
        for pair in combinations( term_rankings[topic_index], 2 ):
            pair_scores.append( w2v_model.similarity(pair[0], pair[1]) )
        # get the mean for all pairs in this topic
        topic_score = sum(pair_scores) / len(pair_scores)
        overall_coherence += topic_score
    # get the mean score across all topics
    return overall_coherence / len(term_rankings)

#### Define a function to get the topic descriptor ( the 5 weighest terms) for each topic:

In [7]:
def get_descriptor( all_terms, H, topic_index):
    # sort indices by descending order
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    top_terms = []
    for t in range(5):
        term_index = top_indices[t]
        top_terms.append( all_terms[term_index] )
        term_weight = H[topic_index,term_index]
    return top_terms

#### Now process each of the models for the triplets (*K*,*$\alpha$*, *$l_1$*) and compute the coherence measure:

In [8]:
func = lambda x: round(x,2)

In [9]:
k_values = []
alpha_values =[]
l1_values =[]
coherences = []
for (k,alpha,l1,W,H,err) in topic_models:
    # Get all of the topic descriptors - the term_rankings, based on top 5 terms
    term_rankings = []
    for topic_index in range(k):
        top_terms = get_descriptor( tfidf_feature_names, H, topic_index)
        term_rankings.append(top_terms)
    # Save each parameter in the corresponding list
    k_values.append(k)
    alpha_values.append(alpha)
    l1_values.append(l1)
    # Now calculate the coherence based on our Word2vec model
    coherences.append( calculate_coherence( w2v_model, term_rankings ) )
    print("K=%02d:  Alpha=%.2f l1=%.1f Coherence=%.4f" % (k,alpha,l1,coherences[-1] ) )

K=02:  Alpha=0.00 l1=0.0 Coherence=0.2804
K=02:  Alpha=0.00 l1=0.1 Coherence=0.2804
K=02:  Alpha=0.00 l1=0.2 Coherence=0.2804
K=02:  Alpha=0.00 l1=0.3 Coherence=0.2804
K=02:  Alpha=0.00 l1=0.4 Coherence=0.2804
K=02:  Alpha=0.00 l1=0.5 Coherence=0.2804
K=02:  Alpha=0.00 l1=0.6 Coherence=0.2804
K=02:  Alpha=0.00 l1=0.7 Coherence=0.2804
K=02:  Alpha=0.00 l1=0.8 Coherence=0.2804
K=02:  Alpha=0.00 l1=0.9 Coherence=0.2804
K=02:  Alpha=0.00 l1=1.0 Coherence=0.2804
K=02:  Alpha=0.01 l1=0.0 Coherence=0.2804
K=02:  Alpha=0.01 l1=0.1 Coherence=0.2804
K=02:  Alpha=0.01 l1=0.2 Coherence=0.2804
K=02:  Alpha=0.01 l1=0.3 Coherence=0.2804
K=02:  Alpha=0.01 l1=0.4 Coherence=0.2804
K=02:  Alpha=0.01 l1=0.5 Coherence=0.2804
K=02:  Alpha=0.01 l1=0.6 Coherence=0.2804
K=02:  Alpha=0.01 l1=0.7 Coherence=0.2804
K=02:  Alpha=0.01 l1=0.8 Coherence=0.2804
K=02:  Alpha=0.01 l1=0.9 Coherence=0.2804
K=02:  Alpha=0.01 l1=1.0 Coherence=0.2804
K=02:  Alpha=0.02 l1=0.0 Coherence=0.2804
K=02:  Alpha=0.02 l1=0.1 Coherence

K=02:  Alpha=0.23 l1=0.4 Coherence=0.2734
K=02:  Alpha=0.23 l1=0.5 Coherence=0.3096
K=02:  Alpha=0.23 l1=0.6 Coherence=0.3096
K=02:  Alpha=0.23 l1=0.7 Coherence=0.3096
K=02:  Alpha=0.23 l1=0.8 Coherence=0.3096
K=02:  Alpha=0.23 l1=0.9 Coherence=0.3096
K=02:  Alpha=0.23 l1=1.0 Coherence=0.3096
K=02:  Alpha=0.24 l1=0.0 Coherence=0.2804
K=02:  Alpha=0.24 l1=0.1 Coherence=0.2804
K=02:  Alpha=0.24 l1=0.2 Coherence=0.2804
K=02:  Alpha=0.24 l1=0.3 Coherence=0.2332
K=02:  Alpha=0.24 l1=0.4 Coherence=0.2734
K=02:  Alpha=0.24 l1=0.5 Coherence=0.3096
K=02:  Alpha=0.24 l1=0.6 Coherence=0.3096
K=02:  Alpha=0.24 l1=0.7 Coherence=0.3096
K=02:  Alpha=0.24 l1=0.8 Coherence=0.3096
K=02:  Alpha=0.24 l1=0.9 Coherence=0.3096
K=02:  Alpha=0.24 l1=1.0 Coherence=0.3096
K=02:  Alpha=0.25 l1=0.0 Coherence=0.2804
K=02:  Alpha=0.25 l1=0.1 Coherence=0.2804
K=02:  Alpha=0.25 l1=0.2 Coherence=0.2804
K=02:  Alpha=0.25 l1=0.3 Coherence=0.2332
K=02:  Alpha=0.25 l1=0.4 Coherence=0.2734
K=02:  Alpha=0.25 l1=0.5 Coherence

K=03:  Alpha=0.03 l1=0.4 Coherence=0.3648
K=03:  Alpha=0.03 l1=0.5 Coherence=0.3648
K=03:  Alpha=0.03 l1=0.6 Coherence=0.3648
K=03:  Alpha=0.03 l1=0.7 Coherence=0.3648
K=03:  Alpha=0.03 l1=0.8 Coherence=0.3648
K=03:  Alpha=0.03 l1=0.9 Coherence=0.3648
K=03:  Alpha=0.03 l1=1.0 Coherence=0.3367
K=03:  Alpha=0.04 l1=0.0 Coherence=0.3648
K=03:  Alpha=0.04 l1=0.1 Coherence=0.3648
K=03:  Alpha=0.04 l1=0.2 Coherence=0.3648
K=03:  Alpha=0.04 l1=0.3 Coherence=0.3648
K=03:  Alpha=0.04 l1=0.4 Coherence=0.3648
K=03:  Alpha=0.04 l1=0.5 Coherence=0.3648
K=03:  Alpha=0.04 l1=0.6 Coherence=0.3648
K=03:  Alpha=0.04 l1=0.7 Coherence=0.3648
K=03:  Alpha=0.04 l1=0.8 Coherence=0.3367
K=03:  Alpha=0.04 l1=0.9 Coherence=0.3367
K=03:  Alpha=0.04 l1=1.0 Coherence=0.3367
K=03:  Alpha=0.05 l1=0.0 Coherence=0.3648
K=03:  Alpha=0.05 l1=0.1 Coherence=0.3648
K=03:  Alpha=0.05 l1=0.2 Coherence=0.3648
K=03:  Alpha=0.05 l1=0.3 Coherence=0.3648
K=03:  Alpha=0.05 l1=0.4 Coherence=0.3648
K=03:  Alpha=0.05 l1=0.5 Coherence

K=03:  Alpha=0.40 l1=0.8 Coherence=0.4185
K=03:  Alpha=0.40 l1=0.9 Coherence=0.4185
K=03:  Alpha=0.40 l1=1.0 Coherence=0.4007
K=03:  Alpha=0.50 l1=0.0 Coherence=0.3648
K=03:  Alpha=0.50 l1=0.1 Coherence=0.3636
K=03:  Alpha=0.50 l1=0.2 Coherence=0.3817
K=03:  Alpha=0.50 l1=0.3 Coherence=0.3854
K=03:  Alpha=0.50 l1=0.4 Coherence=0.3931
K=03:  Alpha=0.50 l1=0.5 Coherence=0.3833
K=03:  Alpha=0.50 l1=0.6 Coherence=0.4185
K=03:  Alpha=0.50 l1=0.7 Coherence=0.4007
K=03:  Alpha=0.50 l1=0.8 Coherence=0.4007
K=03:  Alpha=0.50 l1=0.9 Coherence=0.4007
K=03:  Alpha=0.50 l1=1.0 Coherence=0.4257
K=03:  Alpha=0.60 l1=0.0 Coherence=0.3648
K=03:  Alpha=0.60 l1=0.1 Coherence=0.3636
K=03:  Alpha=0.60 l1=0.2 Coherence=0.3854
K=03:  Alpha=0.60 l1=0.3 Coherence=0.3931
K=03:  Alpha=0.60 l1=0.4 Coherence=0.3833
K=03:  Alpha=0.60 l1=0.5 Coherence=0.4185
K=03:  Alpha=0.60 l1=0.6 Coherence=0.4007
K=03:  Alpha=0.60 l1=0.7 Coherence=0.4007
K=03:  Alpha=0.60 l1=0.8 Coherence=0.3809
K=03:  Alpha=0.60 l1=0.9 Coherence

K=04:  Alpha=0.15 l1=0.7 Coherence=0.3943
K=04:  Alpha=0.15 l1=0.8 Coherence=0.3971
K=04:  Alpha=0.15 l1=0.9 Coherence=0.4151
K=04:  Alpha=0.15 l1=1.0 Coherence=0.4209
K=04:  Alpha=0.16 l1=0.0 Coherence=0.3997
K=04:  Alpha=0.16 l1=0.1 Coherence=0.3997
K=04:  Alpha=0.16 l1=0.2 Coherence=0.3997
K=04:  Alpha=0.16 l1=0.3 Coherence=0.3997
K=04:  Alpha=0.16 l1=0.4 Coherence=0.3892
K=04:  Alpha=0.16 l1=0.5 Coherence=0.3741
K=04:  Alpha=0.16 l1=0.6 Coherence=0.3741
K=04:  Alpha=0.16 l1=0.7 Coherence=0.3971
K=04:  Alpha=0.16 l1=0.8 Coherence=0.3971
K=04:  Alpha=0.16 l1=0.9 Coherence=0.4151
K=04:  Alpha=0.16 l1=1.0 Coherence=0.4209
K=04:  Alpha=0.17 l1=0.0 Coherence=0.3997
K=04:  Alpha=0.17 l1=0.1 Coherence=0.3997
K=04:  Alpha=0.17 l1=0.2 Coherence=0.3997
K=04:  Alpha=0.17 l1=0.3 Coherence=0.3997
K=04:  Alpha=0.17 l1=0.4 Coherence=0.3892
K=04:  Alpha=0.17 l1=0.5 Coherence=0.3741
K=04:  Alpha=0.17 l1=0.6 Coherence=0.3943
K=04:  Alpha=0.17 l1=0.7 Coherence=0.3971
K=04:  Alpha=0.17 l1=0.8 Coherence

...

K=38:  Alpha=0.40 l1=0.2 Coherence=0.5519
K=38:  Alpha=0.40 l1=0.3 Coherence=0.5401
K=38:  Alpha=0.40 l1=0.4 Coherence=0.5281
K=38:  Alpha=0.40 l1=0.5 Coherence=0.5192
K=38:  Alpha=0.40 l1=0.6 Coherence=0.4890
K=38:  Alpha=0.40 l1=0.7 Coherence=0.4409
K=38:  Alpha=0.40 l1=0.8 Coherence=0.4078
K=38:  Alpha=0.40 l1=0.9 Coherence=0.3961
K=38:  Alpha=0.40 l1=1.0 Coherence=0.3725
K=38:  Alpha=0.50 l1=0.0 Coherence=0.5260
K=38:  Alpha=0.50 l1=0.1 Coherence=0.5406
K=38:  Alpha=0.50 l1=0.2 Coherence=0.5428
K=38:  Alpha=0.50 l1=0.3 Coherence=0.5193
K=38:  Alpha=0.50 l1=0.4 Coherence=0.5066
K=38:  Alpha=0.50 l1=0.5 Coherence=0.4697
K=38:  Alpha=0.50 l1=0.6 Coherence=0.4310
K=38:  Alpha=0.50 l1=0.7 Coherence=0.3878
K=38:  Alpha=0.50 l1=0.8 Coherence=0.3813
K=38:  Alpha=0.50 l1=0.9 Coherence=0.3709
K=38:  Alpha=0.50 l1=1.0 Coherence=0.3507
K=38:  Alpha=0.60 l1=0.0 Coherence=0.5238
K=38:  Alpha=0.60 l1=0.1 Coherence=0.5523
K=38:  Alpha=0.60 l1=0.2 Coherence=0.5301
K=38:  Alpha=0.60 l1=0.3 Coherence

K=39:  Alpha=0.12 l1=0.7 Coherence=0.5610
K=39:  Alpha=0.12 l1=0.8 Coherence=0.5551
K=39:  Alpha=0.12 l1=0.9 Coherence=0.5499
K=39:  Alpha=0.12 l1=1.0 Coherence=0.5228
K=39:  Alpha=0.13 l1=0.0 Coherence=0.5329
K=39:  Alpha=0.13 l1=0.1 Coherence=0.5404
K=39:  Alpha=0.13 l1=0.2 Coherence=0.5399
K=39:  Alpha=0.13 l1=0.3 Coherence=0.5485
K=39:  Alpha=0.13 l1=0.4 Coherence=0.5580
K=39:  Alpha=0.13 l1=0.5 Coherence=0.5599
K=39:  Alpha=0.13 l1=0.6 Coherence=0.5526
K=39:  Alpha=0.13 l1=0.7 Coherence=0.5614
K=39:  Alpha=0.13 l1=0.8 Coherence=0.5530
K=39:  Alpha=0.13 l1=0.9 Coherence=0.5450
K=39:  Alpha=0.13 l1=1.0 Coherence=0.5256
K=39:  Alpha=0.14 l1=0.0 Coherence=0.5329
K=39:  Alpha=0.14 l1=0.1 Coherence=0.5404
K=39:  Alpha=0.14 l1=0.2 Coherence=0.5460
K=39:  Alpha=0.14 l1=0.3 Coherence=0.5465
K=39:  Alpha=0.14 l1=0.4 Coherence=0.5560
K=39:  Alpha=0.14 l1=0.5 Coherence=0.5574
K=39:  Alpha=0.14 l1=0.6 Coherence=0.5596
K=39:  Alpha=0.14 l1=0.7 Coherence=0.5437
K=39:  Alpha=0.14 l1=0.8 Coherence

K=39:  Alpha=0.40 l1=0.4 Coherence=0.5183
K=39:  Alpha=0.40 l1=0.5 Coherence=0.5189
K=39:  Alpha=0.40 l1=0.6 Coherence=0.5144
K=39:  Alpha=0.40 l1=0.7 Coherence=0.4500
K=39:  Alpha=0.40 l1=0.8 Coherence=0.4005
K=39:  Alpha=0.40 l1=0.9 Coherence=0.3760
K=39:  Alpha=0.40 l1=1.0 Coherence=0.3633
K=39:  Alpha=0.50 l1=0.0 Coherence=0.5331
K=39:  Alpha=0.50 l1=0.1 Coherence=0.5392
K=39:  Alpha=0.50 l1=0.2 Coherence=0.5438
K=39:  Alpha=0.50 l1=0.3 Coherence=0.5315
K=39:  Alpha=0.50 l1=0.4 Coherence=0.5155
K=39:  Alpha=0.50 l1=0.5 Coherence=0.4856
K=39:  Alpha=0.50 l1=0.6 Coherence=0.4185
K=39:  Alpha=0.50 l1=0.7 Coherence=0.3761
K=39:  Alpha=0.50 l1=0.8 Coherence=0.3809
K=39:  Alpha=0.50 l1=0.9 Coherence=0.3708
K=39:  Alpha=0.50 l1=1.0 Coherence=0.3635
K=39:  Alpha=0.60 l1=0.0 Coherence=0.5281
K=39:  Alpha=0.60 l1=0.1 Coherence=0.5563
K=39:  Alpha=0.60 l1=0.2 Coherence=0.5408
K=39:  Alpha=0.60 l1=0.3 Coherence=0.5135
K=39:  Alpha=0.60 l1=0.4 Coherence=0.4863
K=39:  Alpha=0.60 l1=0.5 Coherence

K=40:  Alpha=0.12 l1=0.5 Coherence=0.5513
K=40:  Alpha=0.12 l1=0.6 Coherence=0.5373
K=40:  Alpha=0.12 l1=0.7 Coherence=0.5334
K=40:  Alpha=0.12 l1=0.8 Coherence=0.5443
K=40:  Alpha=0.12 l1=0.9 Coherence=0.5465
K=40:  Alpha=0.12 l1=1.0 Coherence=0.5407
K=40:  Alpha=0.13 l1=0.0 Coherence=0.5177
K=40:  Alpha=0.13 l1=0.1 Coherence=0.5277
K=40:  Alpha=0.13 l1=0.2 Coherence=0.5239
K=40:  Alpha=0.13 l1=0.3 Coherence=0.5263
K=40:  Alpha=0.13 l1=0.4 Coherence=0.5333
K=40:  Alpha=0.13 l1=0.5 Coherence=0.5466
K=40:  Alpha=0.13 l1=0.6 Coherence=0.5275
K=40:  Alpha=0.13 l1=0.7 Coherence=0.5277
K=40:  Alpha=0.13 l1=0.8 Coherence=0.5478
K=40:  Alpha=0.13 l1=0.9 Coherence=0.5539
K=40:  Alpha=0.13 l1=1.0 Coherence=0.5207
K=40:  Alpha=0.14 l1=0.0 Coherence=0.5177
K=40:  Alpha=0.14 l1=0.1 Coherence=0.5277
K=40:  Alpha=0.14 l1=0.2 Coherence=0.5298
K=40:  Alpha=0.14 l1=0.3 Coherence=0.5351
K=40:  Alpha=0.14 l1=0.4 Coherence=0.5434
K=40:  Alpha=0.14 l1=0.5 Coherence=0.5350
K=40:  Alpha=0.14 l1=0.6 Coherence

K=40:  Alpha=0.40 l1=0.0 Coherence=0.5216
K=40:  Alpha=0.40 l1=0.1 Coherence=0.5403
K=40:  Alpha=0.40 l1=0.2 Coherence=0.5421
K=40:  Alpha=0.40 l1=0.3 Coherence=0.5373
K=40:  Alpha=0.40 l1=0.4 Coherence=0.5074
K=40:  Alpha=0.40 l1=0.5 Coherence=0.5148
K=40:  Alpha=0.40 l1=0.6 Coherence=0.5019
K=40:  Alpha=0.40 l1=0.7 Coherence=0.4431
K=40:  Alpha=0.40 l1=0.8 Coherence=0.3994
K=40:  Alpha=0.40 l1=0.9 Coherence=0.3874
K=40:  Alpha=0.40 l1=1.0 Coherence=0.3653
K=40:  Alpha=0.50 l1=0.0 Coherence=0.5187
K=40:  Alpha=0.50 l1=0.1 Coherence=0.5353
K=40:  Alpha=0.50 l1=0.2 Coherence=0.5316
K=40:  Alpha=0.50 l1=0.3 Coherence=0.5045
K=40:  Alpha=0.50 l1=0.4 Coherence=0.5194
K=40:  Alpha=0.50 l1=0.5 Coherence=0.4804
K=40:  Alpha=0.50 l1=0.6 Coherence=0.4129
K=40:  Alpha=0.50 l1=0.7 Coherence=0.3899
K=40:  Alpha=0.50 l1=0.8 Coherence=0.3806
K=40:  Alpha=0.50 l1=0.9 Coherence=0.3746
K=40:  Alpha=0.50 l1=1.0 Coherence=0.3591
K=40:  Alpha=0.60 l1=0.0 Coherence=0.5181
K=40:  Alpha=0.60 l1=0.1 Coherence

#### Create a dataframe containing all the triplets (*K*,*$\alpha$*, *$l_1$*) and the computed coherence score

In [10]:
dict = {'k_topics' : k_values, 'alpha' : alpha_values, 'l1' : l1_values, 'coherence': coherences}
df = pd.DataFrame(dict)

#### Identify the triplet (*K*,*$\alpha$*, *$l_1$*) maximizing the coherence score

In [11]:
df['coherence'] = pd.to_numeric(df['coherence'])
df.iloc[df['coherence'].idxmax()]

k_topics     39.00000
alpha         0.10000
l1            0.90000
coherence     0.58325
Name: 15992, dtype: float64

### Examine the Final  Model

The analysis  suggests that the most appropriate values are *k=39* topics, *$\alpha$ = 0,10* and *$l_1$= 0,9* for a *coherence score = 0,5831*.

In [12]:
opti_ind = df['coherence'].idxmax()

Store the selected objects

In [13]:
k = df['k_topics'][opti_ind]
alpha = df['alpha'][opti_ind]
l1 = df['l1'][opti_ind]
W = topic_models[opti_ind][3]
H = topic_models[opti_ind][4]

In [14]:
joblib.dump((k,alpha,l1,W,H), "./output/model_selected.pkl") 

['./output/model_selected.pkl']

Display the topic descriptors for this model:

In [15]:
for topic_index in range(k):
    descriptor_terms = get_descriptor( tfidf_feature_names, H, topic_index)
    str_descriptor = ", ".join( descriptor_terms )
    print("Topic %02d: %s" % ( topic_index+1, str_descriptor ) )

Topic 01: energi, effici, consumpt, save, demand
Topic 02: climat, chang, temperatur, futur, project
Topic 03: power, generat, plant, sector, capac
Topic 04: vehicl, fleet, car, hybrid, passeng
Topic 05: industri, cement, sector, product, process
Topic 06: forest, sequestr, wood, sink, stock
Topic 07: steel, iron, product, materi, save
Topic 08: build, residenti, stock, sector, construct
Topic 09: water, resourc, basin, river, irrig
Topic 10: air, pollut, pm2, qualiti, health
Topic 11: electr, generat, demand, grid, suppli
Topic 12: ccs, storag, captur, geolog, plant
Topic 13: transport, sector, road, passeng, freight
Topic 14: polici, develop, econom, countri, use
Topic 15: agricultur, food, product, livestock, farm
Topic 16: tax, revenu, polici, equilibrium, model
Topic 17: heat, pump, district, cool, boiler
Topic 18: intens, structur, consumpt, factor, growth
Topic 19: scenario, model, bau, refer, three
Topic 20: wind, solar, capac, instal, photovolta
Topic 21: cost, abat, option, b